In [1]:
import kafka
import json
import requests
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas as pd



In [2]:
load_dotenv(dotenv_path="E:\LocalProjects\GitRepos\End2EndDataEngineeringPipeline\StockMarketAnalysis\python_api_producer\.env")

True

In [3]:
username = os.getenv('DB_USER')
password = os.getenv('DB_PASS')
database = os.getenv('DB_NAME')
host = os.getenv('DB_HOST')
post = os.getenv('DB_PORT')
print(username)

postgres


In [5]:
path = r"C:\Users\uttka\Downloads\ind_nifty500list.csv"
nifty500 = pd.read_csv(path)
display(nifty500.head())

,Company Name,Industry,Symbol,Series,ISIN Code
0,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038
1,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017
2,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022
3,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025
4,ACME Solar Holdings Ltd.,Power,ACMESOLAR,EQ,INE622W01025


In [10]:
nifty500['created_at'] = pd.Timestamp.now()
display(nifty500)

,Company Name,Industry,Symbol,Series,ISIN Code,created_at
0,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038,2025-09-25 21:38:36.466875
1,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017,2025-09-25 21:38:36.466875
2,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022,2025-09-25 21:38:36.466875
3,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025,2025-09-25 21:38:36.466875
4,ACME Solar Holdings Ltd.,Power,ACMESOLAR,EQ,INE622W01025,2025-09-25 21:38:36.466875
...,...,...,...,...,...,...
496,Zee Entertainment Enterprises Ltd.,Media Entertainment & Publication,ZEEL,EQ,INE256A01028,2025-09-25 21:38:36.466875
497,Zen Technologies Ltd.,Capital Goods,ZENTEC,EQ,INE251B01027,2025-09-25 21:38:36.466875
498,Zensar Technolgies Ltd.,Information Technology,ZENSARTECH,EQ,INE520A01027,2025-09-25 21:38:36.466875
499,Zydus Lifesciences Ltd.,Healthcare,ZYDUSLIFE,EQ,INE010B01027,2025-09-25 21:38:36.466875


#### Insert Nifty 500 Data

In [12]:
con = create_engine(f'postgresql://{username}:{password}@{host}:{post}/{database}')
with con.begin() as conn:
    nifty500.to_sql('stocks_data', con=con, if_exists='replace', index=False)

In [78]:
symbol = 'ADANIENT'
days = '1D'
start_time = '1354410000'
end_time = '1758821896'
base_url = "https://priceapi.moneycontrol.com/techCharts/indianMarket/stock/history?"
params = f"symbol={symbol}&resolution={days}&from={start_time}&to={end_time}&countback=50000&currencyCode=INR"
url = base_url+params
print(url)

https://priceapi.moneycontrol.com/techCharts/indianMarket/stock/history?symbol=ADANIENT&resolution=1D&from=1354410000&to=1758821896&countback=50000&currencyCode=INR


In [79]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'origin': 'https://www.moneycontrol.com'
}

response = requests.get(url, headers=headers)
data = response.json()
display(data)

{'s': 'ok',
 't': [946857600,
  946944000,
  947030400,
  947116800,
  947203200,
  947462400,
  947548800,
  947635200,
  947721600,
  947808000,
  948067200,
  948153600,
  948240000,
  948326400,
  948412800,
  948672000,
  948758400,
  948931200,
  949017600,
  949276800,
  949363200,
  949449600,
  949536000,
  949622400,
  949881600,
  949968000,
  950054400,
  950140800,
  950227200,
  950486400,
  950572800,
  950659200,
  950745600,
  950832000,
  951091200,
  951177600,
  951264000,
  951350400,
  951436800,
  951696000,
  951782400,
  951868800,
  951955200,
  952041600,
  952300800,
  952387200,
  952473600,
  952560000,
  952646400,
  952905600,
  952992000,
  953078400,
  953164800,
  953596800,
  953683200,
  953769600,
  953856000,
  954115200,
  954201600,
  954288000,
  954374400,
  954460800,
  954720000,
  954806400,
  954892800,
  954979200,
  955065600,
  955324800,
  955497600,
  955584000,
  955929600,
  956016000,
  956102400,
  956188800,
  956534400,
  956620

In [ ]:
df = pd.DataFrame({
    'symbol': symbol,
    'metric_time': data['t'],
    'open_price': data['o'],
    'close_price': data['c'],
    'high': data['h'],
    'low': data['l'],
    'volume': data['v'],
    'created_at': pd.Timestamp.now()
})
df['metric_time'] = pd.to_datetime(df['metric_time'], unit='s')
display(df.sort_values("metric_time"))

,symbol,metric_time,open_price,close_price,high,low,volume,created_at
0,ADANIENT,2000-01-03,48.95,49.20,49.20,47.25,276000,2025-09-25 23:08:06.028193
1,ADANIENT,2000-01-04,50.80,49.54,50.80,49.00,344000,2025-09-25 23:08:06.028193
2,ADANIENT,2000-01-05,48.75,48.98,49.00,46.78,24000,2025-09-25 23:08:06.028193
3,ADANIENT,2000-01-06,50.00,51.23,51.25,46.25,1582000,2025-09-25 23:08:06.028193
4,ADANIENT,2000-01-07,47.50,47.20,47.75,47.20,28000,2025-09-25 23:08:06.028193
...,...,...,...,...,...,...,...,...
6374,ADANIENT,2025-09-19,2495.00,2524.00,2579.00,2466.10,7965135,2025-09-25 23:08:06.028193
6375,ADANIENT,2025-09-22,2535.00,2629.50,2659.00,2522.10,7022985,2025-09-25 23:08:06.028193
6376,ADANIENT,2025-09-23,2642.60,2676.40,2695.00,2593.00,5610022,2025-09-25 23:08:06.028193
6377,ADANIENT,2025-09-24,2655.00,2619.80,2676.00,2614.40,1720785,2025-09-25 23:08:06.028193


In [55]:
df.to_csv('test.csv', index=False)